<h2 align="center">NLP: Reconhecimento de produtos</h2>

Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro

Neste script vamos a estudar e aprender uma `tarefa de classificação` usando `Python` e as técnicas de Processamento de Linguagem Natural - clássico (NLP - Clássico).

Como sabemos, a `NLP` é a área de estudo que busca fazer com que a máquina possa compreender e simular a linguagem humana. Em outras palavras, é fazer o computador receber uma frase e ter uma "opinião" sobre ela. Alguns exemplos de aplicação:

* Classificação de texto

* Análise de sentimentos

* Chatbots e assistentes virtuais, etc.



Links de estudo:

* [Streamlit: Predição de flores Iris](https://acervolima.com/implante-um-modelo-de-aprendizado-de-maquina-usando-a-biblioteca-streamlit/)

* [Streamlit: Análise de estoque de produtos ](https://www.alura.com.br/artigos/streamlit-compartilhando-sua-aplicacao-de-dados-sem-dor-de-cabeca)

* [Análise de produtos de supermercado](https://dadosaocubo.com/nlp-com-scikit-learn/)

* [Streamlit: produtos de supermercado](https://dadosaocubo.com/modelos-em-producao-com-streamlit/)

# Importando as bibliotecas necessárias

A seguir temos as bibliotecas que nos ajudará nesta tarefa de classificação. Vamos usar a plataforma [NLTK - Natural Language ToolKit](https://www.nltk.org/) para construir programas Python para trabalhar com dados de linguagem humana, uso de expressões regulares (`REGEX`), etc.  

In [1]:
import numpy as np
import pandas as pd

import nltk
from wordcloud import WordCloud
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline


<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/eddygiusepe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/eddygiusepe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Carregando nosso Dataset e preparando ele 

Sabemos que a preparação de Dados é muito importante para qualquer modelo de Machine Learning. Para as tarefas de NLP não é diferente, um bom tratamento do texto pode levar nosso modelo a resultados incríveis. 

In [2]:
# Carregando nosso Dataset
df = pd.read_csv('https://raw.githubusercontent.com/dadosaocubo/nlp/master/base_mercadologica.csv')
df.head(5)

,descricao,departamento
0,"PASTA INT VITAPOWER 1,005KG AMEND/SHOT",MERCEARIA DOCE
1,ESPONJA BETTANIN BRILHUS C/1,CUIDADOS COM A COZINHA
2,AGUA MIN SCHIN S/GAS 500ML,BEBIDAS NÃO ALCOÓLICAS
3,FITA DUPLA FACE C/SUPORTE SCOTCH,PAPELARIA
4,MASSA PIZZA ROMANHA OREGANO PCT 160G,MASSAS FRESCAS


In [3]:
df.shape

(22009, 2)

In [4]:
# Tipos de departamentos:
df['departamento'].unique()

array(['MERCEARIA DOCE', 'CUIDADOS COM A COZINHA',
       'BEBIDAS NÃO ALCOÓLICAS', 'PAPELARIA', 'MASSAS FRESCAS',
       'BEBIDAS ALCOÓLICAS', 'CUIDADOS COM O CORPO', 'CONGELADOS DOCES',
       'MERCEARIA SALGADA', 'CUIDADOS COM O CABELO',
       'UTILIDADES DE COZINHA', 'CUIDADOS COM A ROUPA', 'AÇOUGUE',
       'CALÇADO', 'ELETRO', 'PET SHOP', 'GORDUROSOS', 'HORTIFRUTI',
       'DESCARTÁVEIS', 'HIGIENE PESSOAL', 'EMPÓRIO',
       'CUIDADOS COM A CASA', 'BRINQUEDO', 'CAMA, MESA E BANHO',
       'SALGADO', 'LAVANDERIA', 'JARDINAGEM', 'HIGIENE PESSOAL INFANTIL',
       'FRIOS', 'PADARIA', 'DECORAÇÃO', 'MANUTENÇÃO', 'LAZER/CAMPING',
       'RESFRIADOS', 'CUIDADOS COM MÃOS E PÉS', 'CUIDADOS COM O BANHEIRO',
       'CUIDADOS COM O ROSTO', 'EMBUTIDOS', 'LEITE',
       'CONGELADOS SALGADOS', 'HIGIENE ORAL', 'TEXTIL', 'PRATOS PRONTOS',
       'CUIDADOS COM A BARBA', 'IOGURTES', 'TABACO', 'INFANTIL',
       'PEIXARIA', 'AUTOMOTIVO', 'MALAS/NECESSAIRES',
       'UTILIDADES DE BANHEIRO', 'SALGAD

In [5]:
# Quantidade de departamentos
len(df['departamento'].unique())

53

In [6]:
# Criamos uma nova coluna
df['nova_descricao'] = df['descricao'].copy()

In [7]:
# Verificamos essa nova coluna
df.head()

,descricao,departamento,nova_descricao
0,"PASTA INT VITAPOWER 1,005KG AMEND/SHOT",MERCEARIA DOCE,"PASTA INT VITAPOWER 1,005KG AMEND/SHOT"
1,ESPONJA BETTANIN BRILHUS C/1,CUIDADOS COM A COZINHA,ESPONJA BETTANIN BRILHUS C/1
2,AGUA MIN SCHIN S/GAS 500ML,BEBIDAS NÃO ALCOÓLICAS,AGUA MIN SCHIN S/GAS 500ML
3,FITA DUPLA FACE C/SUPORTE SCOTCH,PAPELARIA,FITA DUPLA FACE C/SUPORTE SCOTCH
4,MASSA PIZZA ROMANHA OREGANO PCT 160G,MASSAS FRESCAS,MASSA PIZZA ROMANHA OREGANO PCT 160G


Nessa nova coluna vamos tratar a pontuação, acentuação, caracteres especiais, palavras vazias (`stop words`) e números, que são caracteres que interferem na análise do texto. Em nosso caso a pontuação não é tão relevante para a identificação de um produto, mas MUITO CUIDADO em outro contexto. <font color="orange">Por exemplo em análsie de sentimentos</font>: quando um texto contém  `?` e quando não contém `?`. 

Começamos com a puntuação e acentuação (aplicamos `regex`):

In [8]:
df['nova_descricao'] = df['nova_descricao'].str.replace('[,.:;!?]+', ' ', regex=True).copy()

In [16]:
# Vejamos os cambios
df.head(6)

,descricao,departamento,nova_descricao
0,"PASTA INT VITAPOWER 1,005KG AMEND/SHOT",MERCEARIA DOCE,PASTA INT VITAPOWER 1 005KG AMEND/SHOT
1,ESPONJA BETTANIN BRILHUS C/1,CUIDADOS COM A COZINHA,ESPONJA BETTANIN BRILHUS C/1
2,AGUA MIN SCHIN S/GAS 500ML,BEBIDAS NÃO ALCOÓLICAS,AGUA MIN SCHIN S/GAS 500ML
3,FITA DUPLA FACE C/SUPORTE SCOTCH,PAPELARIA,FITA DUPLA FACE C/SUPORTE SCOTCH
4,MASSA PIZZA ROMANHA OREGANO PCT 160G,MASSAS FRESCAS,MASSA PIZZA ROMANHA OREGANO PCT 160G
5,BISC LANCH BAUDUCCO RECHEADINHO CHOCOLATE 104G,MERCEARIA DOCE,BISC LANCH BAUDUCCO RECHEADINHO CHOCOLATE 104G


Vamos remover os `caracteres especiais` (#, $, &, /, *, %, etc) na descrição dos itens. Façamos isso com regex, assim:  

In [17]:
df['nova_descricao'] = df['nova_descricao'].str.replace ('[/<>()|\+\-\$%&#@\'\"]+', ' ', regex=True).copy()

In [19]:
# Vejamos, novamente, esses cambios
df.head()

,descricao,departamento,nova_descricao
0,"PASTA INT VITAPOWER 1,005KG AMEND/SHOT",MERCEARIA DOCE,PASTA INT VITAPOWER 1 005KG AMEND SHOT
1,ESPONJA BETTANIN BRILHUS C/1,CUIDADOS COM A COZINHA,ESPONJA BETTANIN BRILHUS C 1
2,AGUA MIN SCHIN S/GAS 500ML,BEBIDAS NÃO ALCOÓLICAS,AGUA MIN SCHIN S GAS 500ML
3,FITA DUPLA FACE C/SUPORTE SCOTCH,PAPELARIA,FITA DUPLA FACE C SUPORTE SCOTCH
4,MASSA PIZZA ROMANHA OREGANO PCT 160G,MASSAS FRESCAS,MASSA PIZZA ROMANHA OREGANO PCT 160G


Em problemas de classificação de texto, os números podem confundir os classifadores, diminuindo a assertividade dos mesmos. A seguir vamos a remover os números, assim:

In [20]:
df['nova_descricao'] = df['nova_descricao'].str.replace('[0-9]+', '', regex=True).copy()

In [21]:
df.head()

,descricao,departamento,nova_descricao
0,"PASTA INT VITAPOWER 1,005KG AMEND/SHOT",MERCEARIA DOCE,PASTA INT VITAPOWER KG AMEND SHOT
1,ESPONJA BETTANIN BRILHUS C/1,CUIDADOS COM A COZINHA,ESPONJA BETTANIN BRILHUS C
2,AGUA MIN SCHIN S/GAS 500ML,BEBIDAS NÃO ALCOÓLICAS,AGUA MIN SCHIN S GAS ML
3,FITA DUPLA FACE C/SUPORTE SCOTCH,PAPELARIA,FITA DUPLA FACE C SUPORTE SCOTCH
4,MASSA PIZZA ROMANHA OREGANO PCT 160G,MASSAS FRESCAS,MASSA PIZZA ROMANHA OREGANO PCT G


As `palavras vazias` ou `stop words`, como são mais conhecidas, são palavras que não agregam sentidos as frases, <font color="yellow">por exemplo:</font> `conjunções`, `artigos` e `pronomes`. Não existe uma lista definitiva com essas palavras, como sempre essa análise vai depender do problema.


Vou mostrar a lista de `stop words` em `português` da biblioteca `NLTK`, que pode ser uma base inicial de partida.

In [26]:
# Lista com as 50 primeiras stop words da biblioteca NLTK
str(stopwords.words('portuguese')[:100])

"['a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'às', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'éramos', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estou', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'formos', 'fosse', 'fossem', 'fôssemos', 'fui', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'haver', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam']"

In [27]:
# Devido a nosso problema ser particular, as palavras da biblioteca NLTK não abrange nossos itens 
# Criamos um objeto do tipo lista com todas as nossas stop words
stop_words = ['em','sao','ao','de','da','do','para','c','kg','un',
              'ml','pct','und','das','no','ou','pc','gr','pt','cm',
              'vd','com','sem','gfa','jg','la','1','2','3','4','5',
              '6','7','8','9','0','a','b','c','d','e','lt','f','g',
              'h','i','j','k','l','m','n','o','p','q','r','s','t',
              'u','v','x','w','y','z']

Agora vamos a entender a `tokenização` e a transformação desses `tokens` em vetores, para que possa ser entendido por nosso algoritmo. Basicamente, a tokenização é a forma de separação do texto em tokens, ou seja, decompor o texto por cada termo. 

A seguir mostramos um exemplo simples:

In [30]:
frase = 'Sou estudante da UFES e faço parte do laboratório de pesquisa LabVISIO, na Engenharia Elétrica.'
nltk.word_tokenize(frase)
 

['Sou',
 'estudante',
 'da',
 'UFES',
 'e',
 'faço',
 'parte',
 'do',
 'laboratório',
 'de',
 'pesquisa',
 'LabVISIO',
 ',',
 'na',
 'Engenharia',
 'Elétrica',
 '.']

In [31]:
# Criação da função CountVectorizer do scikit-learn.
cvt = CountVectorizer(strip_accents='ascii', 
                      lowercase=True, 
                      stop_words=stop_words)

In [32]:
import numpy as np

# Seleção de dois itens da base de dados
exemplo_descricao = df['nova_descricao'][:2]
exemplo_descricao.values
 
np.array(['PASTA INT VITAPOWER  KG AMEND SHOT',
       'ESPONJA BETTANIN BRILHUS C '], dtype=object)
 
# Transformação dos dois itens em vetores binários
exemplo_descricao_cvt = cvt.fit_transform(exemplo_descricao)
exemplo_descricao_cvt.toarray()
 
np.array([[1, 0, 0, 0, 1, 1, 1, 1],
       [0, 1, 1, 1, 0, 0, 0, 0]])

array([[1, 0, 0, 0, 1, 1, 1, 1],
       [0, 1, 1, 1, 0, 0, 0, 0]])

In [33]:
X_cvt = cvt.fit_transform(df['nova_descricao'])

In [34]:
type(X_cvt)

scipy.sparse._csr.csr_matrix

In [35]:
# Criação da função TfidfTransformer
tfi = TfidfTransformer(use_idf=True)

In [36]:
# Transformação dos exemplos com a normalização tf-idf
exemplo_descricao_tfi = tfi.fit_transform(exemplo_descricao_cvt)
exemplo_descricao_tfi.toarray()

array([[0.4472136 , 0.        , 0.        , 0.        , 0.4472136 ,
        0.4472136 , 0.4472136 , 0.4472136 ],
       [0.        , 0.57735027, 0.57735027, 0.57735027, 0.        ,
        0.        , 0.        , 0.        ]])

In [37]:
X_tfi = tfi.fit_transform(X_cvt)

In [38]:
type(X_tfi)

scipy.sparse._csr.csr_matrix

In [39]:
# A entrada será a transformação de vetores com a normalização tf-idf
entrada = X_tfi
# A saida será os departamentos
saida = df['departamento']
# Separando 20% dos dados para teste
X_train, X_test, y_train, y_test = train_test_split(entrada, 
                                                    saida, 
                                                    test_size=0.2)

In [40]:
# Criando modelo
clf = LinearSVC()
# Treinamento do modelo
clf.fit(X_train, y_train)

LinearSVC()

In [44]:
# Realizando a predição
resultado = clf.predict(X_test)
# Avaliando o modelo
print('Acurácia: {:.2f}'.format(metrics.accuracy_score(y_test, resultado)))


Acurácia: 0.95


<font color="orange">Siga o seguinte exemplo, para salvar os modelos que treinei neste script</font>

In [50]:
# Salvando nosso Modelo

#from sklearn.externals import joblib  <-- Isto dá ERRO não usar
import joblib 
from joblib import dump, load

dump(clf, 'models/clfEddy.joblib')

['models/clfEddy.joblib']

# Usando o modelo

In [38]:
def novo_item(descricao):
  novo_cvt = cvt.transform(pd.Series(descricao))
  novo_tfi = tfi.transform(novo_cvt)
  departamento = clf.predict(novo_tfi)[0]
  return departamento

In [42]:
# Lista de exemplos de novos produtos
itens = ['FEIJÃO','AÇÚCAR','ALFACE']
# Loop for para fazer a predição do departamento de novos produtos
for item in itens:
  print('Produto:', item, 'Departamento:', novo_item(item))

Produto: FEIJÃO Departamento: MERCEARIA SALGADA
Produto: AÇÚCAR Departamento: MERCEARIA DOCE
Produto: ALFACE Departamento: HORTIFRUTI
